In [ ]:
%pip install onnx onnxruntime-gpu torchaudio scikit-learn transformers
%pip install soundfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
%pip install tokenizers
%pip install resampy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.2 MB/s eta 0:00:00


# ONNX download

*   VGGish model을 load
*   wav 파일을 전처리(mel spectrum으로)

In [ ]:
cd "g:\내 드라이브\IAP_Final/PC/source"

g:\내 드라이브\IAP_Final\source


c:\Users\JUNE\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os
import urllib.request
import sys

def show_progress(block_num, block_size, total_size):
    downloaded = block_num * block_size
    percent = min(downloaded / total_size * 100, 100)
    bar = '█' * int(percent // 2) + '-' * (50 - int(percent // 2))
    sys.stdout.write(f"\r [{bar}] {percent:.2f}%")
    sys.stdout.flush()

url = "https://essentia.upf.edu/models/feature-extractors/vggish/audioset-vggish-3.onnx"
save_path = "./model/VGGish/audioset-vggish-3.onnx"

# 디렉토리가 없으면 생성
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# 파일이 없을 때만 다운로드
if not os.path.exists(save_path):
    print("📥 모델을 다운로드 중...")
    urllib.request.urlretrieve(url, save_path, reporthook=show_progress)
    print("✅ 다운로드 완료:", save_path)
else:
    print("✅ 이미 파일이 존재합니다. 다운로드 생략:", save_path)


✅ 이미 파일이 존재합니다. 다운로드 생략: ./onnx/audioset-vggish-3.onnx


In [ ]:
import torch
import torchaudio
import torchaudio.transforms as T

def preprocess_for_vggish(wav_path, target_sample_rate=16000):
    # 오디오 로드 및 리샘플링
    waveform, sr = torchaudio.load(wav_path)
    if sr != target_sample_rate:
        resampler = T.Resample(sr, target_sample_rate)
        waveform = resampler(waveform)

    # 채널 평균 (Mono로 변환)
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # Mel Spectrogram (torchaudio의 설정은 VGGish에 맞게)
    mel_transform = T.MelSpectrogram(
        sample_rate=target_sample_rate,
        n_fft=400,
        win_length=400,
        hop_length=160,
        n_mels=64,
        f_min=125,
        f_max=7500,
        power=2.0
    )
    mel = mel_transform(waveform)  # (1, 64, T)

    # dB 변환
    db_transform = T.AmplitudeToDB()
    mel_db = db_transform(mel)  # (1, 64, T)

    # Frame 단위 자르기 (96x64)
    mel_db = mel_db.squeeze(0).transpose(0, 1)  # (T, 64)
    if mel_db.shape[0] < 96:
        # 패딩
        pad_len = 96 - mel_db.shape[0]
        mel_db = torch.nn.functional.pad(mel_db, (0, 0, 0, pad_len))
    else:
        mel_db = mel_db[:96, :]  # 앞에서부터 96 프레임 자름
    mel_db = mel_db.transpose(0, 1)
    mel_db = mel_db.unsqueeze(0)  # (1, 1, 96, 64)
    return mel_db


# recommend_iap.py
Jetson Nano에서 사용자 음성 .wav 파일을 불러오고, ONNX 모델로 벡터를 추출 후 노래 recommend

In [ ]:
import os
import torchaudio
import shutil
import onnxruntime
import numpy as np

In [ ]:

def run_vggish_onnx(sess, mel_db):
    """
    mel_db: torch.Tensor of shape (1, 1, 96, 64)
    onnx_path: path to vggish.onnx
    """

    mel_spec_np = mel_db.numpy().astype(np.float32)

    # 입력 이름 확인
    input_name = sess.get_inputs()[0].name
    output_name = sess.get_outputs()[0].name

    # 추론
    outputs = sess.run([output_name], {input_name: mel_spec_np})
    embedding = outputs[0]  # shape: (1, 128)

    return embedding.squeeze()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/IAP_Final/PC/source'

/content/drive/MyDrive/IAP_Final/source


In [ ]:
cd "g:\내 드라이브\IAP_Final/PC/source"

g:\내 드라이브\IAP_Final\source


c:\Users\JUNE\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os
import shutil
import numpy as np
import torchaudio
import onnxruntime
import re

def embedding(input_dir = "musics/highlight_clips", output_npz = "musics/music_vectors.npz", key_path = "musics/song_list.txt"):
# 설정
    onnx_path = "model/VGGish/audioset-vggish-3.onnx"

    # 기존 벡터 불러오기 (있다면)
    audio_vectors = {}
    if os.path.exists(output_npz):
        existing = np.load(output_npz, allow_pickle=True)
        audio_vectors.update({k: existing[k] for k in existing.files})
        print(f"✅ 기존 벡터 {len(audio_vectors)}개 로드됨")

    # ONNX 세션 생성
    sess = onnxruntime.InferenceSession(
        onnx_path,
        providers=["CUDAExecutionProvider", "CPUExecutionProvider"]
    )


    # 벡터 추출 루프
    def order_key(reference_list_path=None):
        reference_order = {}
        if reference_list_path and os.path.exists(reference_list_path):
            with open(reference_list_path, "r", encoding="utf-8") as f:
                titles = [line.strip() for line in f]
                for idx, title in enumerate(titles):
                    filename = f"cliped_{title}.wav"
                    reference_order[filename] = idx

        def sort_key(filename):
            if filename in reference_order:
                return reference_order[filename]
            match = re.search(r"\((\d+)\)", filename)
            return int(match.group(1)) if match else float('inf')

        return sort_key

    for fname in sorted(os.listdir(input_dir), key=order_key(key_path)):
        if not fname.lower().endswith(".wav"):
            continue

        input_path = os.path.join(input_dir, fname)

        try:
            key = os.path.splitext(fname)[0]

            mel_db = preprocess_for_vggish(input_path, target_sample_rate=16000)
            audio_vectors[key] = run_vggish_onnx(sess, mel_db)

            moved_dir = "./"+input_dir+"/loaded_musics"
            os.makedirs(moved_dir, exist_ok=True)
            shutil.move(input_path, os.path.join(moved_dir, fname))

            print(f"✅ 처리 완료: {fname}")

        except Exception as e:
            print(f"❌ 실패: {fname} - {e}")

    # .npz 저장
    np.savez(output_npz, **audio_vectors)
    print(f"\n💾 총 {len(audio_vectors)}개의 벡터 저장 완료 → {output_npz}")


In [ ]:
embedding()
embedding(input_dir = "voices/user_voice_clips", output_npz = "voices/voice_vectors.npz")

✅ 처리 완료: cliped_헤어지자 말해요 - 박재정.wav
✅ 처리 완료: cliped_좋은 밤 좋은 꿈 - 너드커넥션.wav
✅ 처리 완료: cliped_소녀 - 오혁.wav
✅ 처리 완료: cliped_어디에도 - 엠씨더맥스.wav
✅ 처리 완료: cliped_새벽에 걸려온 너의 전화는 - 한동근.wav
✅ 처리 완료: cliped_여전히 아름다운지 - 토이.wav
✅ 처리 완료: cliped_가시 - 버즈.wav
✅ 처리 완료: cliped_응급실 - izi.wav
✅ 처리 완료: cliped_겁쟁이 - 버즈.wav
✅ 처리 완료: cliped_너의 모든 순간 - 성시경.wav
✅ 처리 완료: cliped_아무노래 - 지코.wav
✅ 처리 완료: cliped_자니 - 다이나믹듀오.wav
✅ 처리 완료: cliped_예쁘잖아 - 기리보이.wav
✅ 처리 완료: cliped_Trip - 릴러말즈.wav
✅ 처리 완료: cliped_시간이 들겠지 - 로꼬.wav
✅ 처리 완료: cliped_어땠을까 - 싸이.wav
✅ 처리 완료: cliped_에라 모르겠다 - 빅뱅.wav
✅ 처리 완료: cliped_어떻게 지내 - 오반.wav
✅ 처리 완료: cliped_예뻤어 - 데이식스.wav
✅ 처리 완료: cliped_나에게로 떠나는 여행 - 버즈.wav
✅ 처리 완료: cliped_질풍가도 - 유정석.wav
✅ 처리 완료: cliped_주저하는 연인들을 위해 - 잔나비.wav
✅ 처리 완료: cliped_고백 - 델리스파이스.wav
✅ 처리 완료: cliped_밤이 깊었네 - 크라잉넛.wav
✅ 처리 완료: cliped_깊은 밤을 날아서 - 이문세.wav
✅ 처리 완료: cliped_비행기 - 거북이.wav
✅ 처리 완료: cliped_Welcome to the Show - 데이식스.wav
✅ 처리 완료: cliped_시작 - 가호.wav
✅ 처리 완료: cliped_FANTASTIC BABY - 빅뱅.wav
✅ 처리 완료: cliped_뱅뱅뱅 - 빅뱅.wav


In [ ]:
import os

def normalize_dashes_in_filenames(folder_path):
    # 바꿀 문자 정의
    replacements = {
        "–": "-",  # 엔대시 (U+2013)
        "−": "-",  # 마이너스 기호 (U+2212)
        "—": "-",  # 엠대시 (U+2014)
    }

    for filename in os.listdir(folder_path):
        new_name = filename
        for old, new in replacements.items():
            new_name = new_name.replace(old, new)

        if new_name != filename:
            src = os.path.join(folder_path, filename)
            dst = os.path.join(folder_path, new_name)
            os.rename(src, dst)
            print(f"✅ renamed: {filename} → {new_name}")

# 사용 예시
normalize_dashes_in_filenames("musics/highlight_clips")
normalize_dashes_in_filenames("musics/original_musics")


npz -> csv (ppt에 넣을 표 만들기)

In [ ]:
import numpy as np
import pandas as pd

def npz_to_csv(npz_path="y_vectors.npz", csv_path="y_vectors.csv"):
    data = np.load(npz_path)
    rows = []

    for fname in data.files:
        vector = data[fname]
        row = [fname + ".wav"] + vector.tolist()
        rows.append(row)

    df = pd.DataFrame(rows)
    df.columns = ["filename"] + [f"dim_{i}" for i in range(len(rows[0]) - 1)]
    df.to_csv(csv_path, index=False)
    print(f"✅ CSV 저장 완료: {csv_path}")


In [ ]:
npz_to_csv()

✅ CSV 저장 완료: y_vectors.csv
